### Задачи

Скачайте один любой ноутбук из датасета и выполните следующие задачи:
- Найдите и выведите версию юпитер ноутбука и питона.
- Создайте таблицу, где будут следующие колонки: cell_id ,тип ячейки (code/markdown/other) и контент ячейки
- Для ячеек с маркдауном определите язык текста
- Для ячеек с кодом определите количество вызовов функций в ячейке

### Установка и импорт библиотек

In [47]:
!pip install polyglot

In [48]:
!pip install PyICU

In [49]:
!pip install pycld2

In [50]:
from urllib.request import urlretrieve
import pandas as pd
import json
import ast

from polyglot.detect import Detector
from polyglot.detect.base import logger as polyglot_logger

In [51]:
pd.options.mode.chained_assignment = None
pd.set_option("max_colwidth", 80)

polyglot_logger.setLevel("ERROR")

### Скачивание ноутбука для анализа

Команда Datalore by JetBrains провела анализ нескольких миллионов публично доступных репозиториев Github с Jupyter-ноутбуками. Датасет из 9,72 миллиона файлов доступен для скачивания:

- JSON с именами файлов jupyter ноутбуков https://github-notebooks-samples.s3-eu-west-1.amazonaws.com/ntbslist.json

- Для получения прямой ссылки можно добавить имя файла из JSON к адресу бакета, например: https://github-notebooks-update1.s3-eu-west-1.amazonaws.com/0000137e4ae8e20431accee8cd5e54c609061846.ipynb

In [52]:
notebook_url = 'https://github-notebooks-update1.s3-eu-west-1.amazonaws.com/0000137e4ae8e20431accee8cd5e54c609061846.ipynb'
notebook_path = 'target_notebook.ipynb'
urlretrieve(notebook_url, notebook_path)

('target_notebook.ipynb', <http.client.HTTPMessage at 0x7fdc67d51510>)

### Базовая структура классов

Для упрощения работы можно использовать классы ячейки и ноутбука, внутри которых определить методы для получения метаданных.

На верхнем уровне абстракции Jupyter ноутбук - это словарь с несколькими ключами:
- metadata (dict)
- nbformat (int)
- nbformat_minor (int)
- cells (list)

Данные о ядре и языке ноутбука можно получить по ключу **metadata**. В свою очередь ячейки со своими метаданными содержатся по ключу **cells**.

[Подробнее](https://ipython.org/ipython-doc/3/notebook/nbformat.html)

In [53]:
class Cell:
  def __init__(self, id, cell_type, content):
    self.content = content
    self.ctype = cell_type
    self.id = id
  

class NoteBook:
  def __init__(self, data):
    self.json_data = data
    self.__obtain_metadata()

  def __obtain_metadata(self):
    self.kernel_name = self.json_data['metadata']['kernelspec']['display_name']
    self.language_version = self.json_data['metadata']['language_info']['version']

    self.cells = []

    for cell_obj in self.json_data['cells']:
      content = ''.join(cell_obj['source'])
      cell = Cell(
          id=cell_obj['metadata']['id'], 
          cell_type=cell_obj['cell_type'], 
          content=content
      )
      self.cells.append(cell)

  def __make_cells_table_cols(self):
    for cell in self.cells:
      yield (cell.id, cell.ctype, cell.content)

  @property
  def kernel_version(self):
    return f'Ядро ноутбука - {self.kernel_name}'

  @property
  def python_version(self):
    return f'Версия python - {self.language_version}'

  @property
  def cells_table(self):
    return pd.DataFrame(
        data=[item for item in self.__make_cells_table_cols()],
         columns=['Id', 'Type', 'Content']
    )

### Версия ядра и python 

In [54]:
with open(notebook_path, "r") as read_file:
    notebook_obj = json.load(read_file)
    nb = NoteBook(notebook_obj)
    print(nb.kernel_version)
    print(nb.python_version)

Ядро ноутбука - Python 3
Версия python - 3.7.4


### Содержимое и тип всех ячеек

In [55]:
cells = nb.cells_table
cells.head()

,Id,Type,Content
0,KV1cRrUadT7_,code,import pystan\nimport pandas as pd\nimport os\nimport re\nimport numpy as np...
1,SVgT--Y4dT8I,markdown,# **Data**\n\n20 students taking CS146 during the Spring 2020 semester compi...
2,jBABL3T4dT8K,code,'''\nData pre-processed in Excel:\nNormalized data;\nManually fixed data ent...
3,SSpe0b4hdT8P,markdown,"After deleting 296 observations with missing prices, we have a final dataset..."
4,jCmzioksdT8R,code,"for col in ['Store', 'Neighborhood', 'Product']:\n df_clean[col] = df_cle..."


### Определение языка для ячеек с типом markdown

In [56]:
def determine_language(content):
  top_lang = Detector(content).languages[0]
  return top_lang.name

In [57]:
markdown_cells = cells[cells['Type'] == 'markdown']
markdown_cells['language'] = markdown_cells['Content'].map(lambda x: determine_language(x))
markdown_cells.head()

,Id,Type,Content,language
1,SVgT--Y4dT8I,markdown,# **Data**\n\n20 students taking CS146 during the Spring 2020 semester compi...,English
3,SSpe0b4hdT8P,markdown,"After deleting 296 observations with missing prices, we have a final dataset...",English
5,jHRlVKExgGR9,markdown,"From the means above, we can see that a single egg is the cheapest product t...",English
7,CMKusirbfNts,markdown,"From the descriptive statistics above, it seems that Supermercados Día has t...",English
9,Y49hpJ5YjdrM,markdown,Comparing the price ranges of Taipei and Buenos Aires neighborhoods points t...,English


### Определение количества вызовов функций в ячейках с кодом

Исходный код в содержимом ячеек представляет собой текст, поэтому для определения количества вызовов функций можно было бы использовать регулярные выражения, или воспользоваться абстрактным синтаксическим деревом (AST). Оно создаётся на стадии синтаксического разбора, обрабатывается путём обхода при проверке семантических правил и проверке/определении типов, а затем также путём обхода AST выполняется генерация кода.

В python существует стандартный модуль ast.

[Подробнее](https://docs.python.org/3/library/ast.html)

In [58]:
class CallParser(ast.NodeVisitor):
    def __init__(self):
        self.fparts = []

    def visit_Attribute(self, node):
        ast.NodeVisitor.generic_visit(self, node)
        self.fparts.append(node.attr)

    def visit_Name(self, node):
        self.fparts.append(node.id)


class FindCalls(ast.NodeVisitor):
  def __init__(self):
        self.fnames = []

  def visit_Call(self, node):
      parser = CallParser()
      parser.visit(node.func)
      self.fnames.append(".".join(parser.fparts))
      ast.NodeVisitor.generic_visit(self, node)

In [59]:
def count_fun_calls(content):
  tree = ast.parse(content)
  fc = FindCalls()
  fc.visit(tree)
  return fc.fnames

In [60]:
code_cells = cells[cells['Type'] == 'code']
code_cells['Functions calls'] = code_cells['Content'].map(lambda x: count_fun_calls(x))
code_cells['Functions calls count'] = code_cells['Functions calls'].map(lambda x: len(x))
code_cells.head()

,Id,Type,Content,Functions calls,Functions calls count
0,KV1cRrUadT7_,code,import pystan\nimport pandas as pd\nimport os\nimport re\nimport numpy as np...,[drive.mount],1
2,jBABL3T4dT8K,code,'''\nData pre-processed in Excel:\nNormalized data;\nManually fixed data ent...,"[pd.read_csv, data.head, df.head, pd.melt, df.rename, df.replace, df.head, p...",15
4,jCmzioksdT8R,code,"for col in ['Store', 'Neighborhood', 'Product']:\n df_clean[col] = df_cle...","[df_clean.col.astype, plt.figure, plt.scatter, plt.show, print, price.groupb...",7
6,Bcqy-vQwfE23,code,"# Store descriptive statistics\nplt.figure(figsize=(15,8))\nplt.scatter(stor...","[plt.figure, plt.scatter, plt.show, print, price.groupby.store.mean, price.g...",6
8,SUC7p9N4fIhw,code,"# Neighborhood descriptive statistics\nplt.figure(figsize=(15,8))\nplt.scatt...","[plt.figure, plt.scatter, plt.show, print, price.groupby.neighborhood.mean, ...",6
